In [81]:
import numpy as np
import pandas as pd
import re
import glob
import os 
from sklearn.model_selection import GridSearchCV,cross_val_score,cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

In [2]:
path_train="../No_window/nw_scores/nw_stack_train/"
path_test="../No_window/nw_scores/nw_stack_test/"

In [3]:
dict_train = {} 
dict_test = {}
train_files = [i for i in os.listdir(path_train)]
test_files = [i for i in os.listdir(path_test)]
for f in train_files:
    dict_train[re.sub("\.csv","",f)] = pd.read_csv(path_train+f)
for f in test_files:
    dict_test[re.sub("\.csv","",f)] = pd.read_csv(path_test+f)

In [4]:
col_no_ID=['class1', 'class2', 'class3', 'class4', 'class5', 'class6', 'class7',
       'class8', 'class9']

In [5]:
#Affecting  columns name
for name in dict_test.keys():
    dict_train[name].columns=[name+"_"+s for s in col_no_ID]+["ID"]
    dict_test[name].columns=[name+"_"+s for s in col_no_ID]+["ID"]

In [6]:
stack_train=dict_train["nw_xgb_d2v"].merge(dict_train["nw_xgb_tfidf"],
on="ID").merge(dict_train["nw_xgb_w2v"],on="ID").merge(dict_train["nw_lgbm_d2v"],
on="ID").merge(dict_train["nw_lgbm_tfidf"],on="ID").merge(dict_train["nw_lgbm_w2v"],
on="ID").merge(dict_train["nw_lr_d2v"],on="ID").merge(dict_train["nw_lr_tfidf"],
on="ID").merge(dict_train["nw_lr_w2v"],on="ID").merge(dict_train["nw_ada_d2v"],
on="ID").merge(dict_train["nw_ada_tfidf"],on="ID").merge(dict_train["nw_ada_w2v"],
on="ID").merge(dict_train["nw_rf_d2v"],on="ID").merge(dict_train["nw_rf_tfidf"],
on="ID").merge(dict_train["nw_rf_w2v"],on="ID")
#.merge(dict_train["nowdw_neural_net"],on="ID")
#a tester 

In [7]:
stack_test=dict_test["nw_xgb_d2v"].merge(dict_test["nw_xgb_tfidf"],
on="ID").merge(dict_test["nw_xgb_w2v"],on="ID").merge(dict_test["nw_lgbm_d2v"],
on="ID").merge(dict_test["nw_lgbm_tfidf"],on="ID").merge(dict_test["nw_lgbm_w2v"],
on="ID").merge(dict_test["nw_lr_d2v"],on="ID").merge(dict_test["nw_lr_tfidf"],
on="ID").merge(dict_test["nw_lr_w2v"],on="ID").merge(dict_test["nw_ada_d2v"],
on="ID").merge(dict_test["nw_ada_tfidf"],on="ID").merge(dict_test["nw_ada_w2v"],
on="ID").merge(dict_test["nw_rf_d2v"],on="ID").merge(dict_test["nw_rf_tfidf"],
on="ID").merge(dict_test["nw_rf_w2v"],on="ID")

In [8]:
#quand le merging est fini : table unique, on peut monter un meta_model

In [28]:
y=pd.read_csv("../bases/new_training_variants.csv").Class.values - 1

In [43]:
ID_test=stack_test["ID"]
ID_train=stack_train["ID"]

In [52]:
meta_feat_train=pd.read_csv("../No_window/nw_meta_features/meta_train_l1l2.csv")
meta_feat_test=pd.read_csv("../No_window/nw_meta_features/meta_test_l1l2.csv")

In [54]:
meta_feat_train=meta_feat_train.drop("Unnamed: 0",axis=1)
meta_feat_test=meta_feat_test.drop("Unnamed: 0",axis=1)

In [55]:
magic_feat=pd.read_csv("../l2_meta_features/svd25_molecular_functions.csv")
magic_feat=magic_feat.drop(["Class","Gene","ID","Variation"],axis=1)
magic_train=magic_feat[:len(y)]
magic_test=magic_feat[len(y):]
magic_train=magic_train.reset_index(drop=True)
magic_test=magic_test.reset_index(drop=True)

In [56]:
stack_train_1=pd.concat((stack_train,meta_feat_train,magic_train),axis=1)
stack_test_1=pd.concat((stack_test,meta_feat_test,magic_test),axis=1)

In [61]:
X_train=np.array(stack_train_1.drop("ID",axis=1))

In [62]:
X_test=np.array(stack_test_1.drop("ID",axis=1))

In [73]:
#Ridge vs Lasso searching
param_test= {
    "C" : [0.1,0.3,0.5,0.7,0.9, 1, 2,3,4,5],
    "penalty": ["l1","l2"]
}


In [74]:
kf = StratifiedKFold(n_splits=5, random_state=26, shuffle=True)

In [75]:
gsearch=GridSearchCV(estimator=LogisticRegression(), param_grid = param_test,scoring="neg_log_loss",n_jobs=4,iid=False, cv=kf)
gsearch.fit(X_train,y)
gsearch.grid_scores_, gsearch.best_params_, gsearch.best_score_


C:\Users\Franck\Documents\Continuum\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.92888, std: 0.04135, params: {'C': 0.1, 'penalty': 'l1'},
  mean: -0.93251, std: 0.04187, params: {'C': 0.1, 'penalty': 'l2'},
  mean: -0.89225, std: 0.04256, params: {'C': 0.3, 'penalty': 'l1'},
  mean: -0.92617, std: 0.04280, params: {'C': 0.3, 'penalty': 'l2'},
  mean: -0.88581, std: 0.04346, params: {'C': 0.5, 'penalty': 'l1'},
  mean: -0.92481, std: 0.04126, params: {'C': 0.5, 'penalty': 'l2'},
  mean: -0.88120, std: 0.04637, params: {'C': 0.7, 'penalty': 'l1'},
  mean: -0.92502, std: 0.04321, params: {'C': 0.7, 'penalty': 'l2'},
  mean: -0.87942, std: 0.04865, params: {'C': 0.9, 'penalty': 'l1'},
  mean: -0.92403, std: 0.04246, params: {'C': 0.9, 'penalty': 'l2'},
  mean: -0.87965, std: 0.04914, params: {'C': 1, 'penalty': 'l1'},
  mean: -0.92379, std: 0.04452, params: {'C': 1, 'penalty': 'l2'},
  mean: -0.88506, std: 0.05983, params: {'C': 2, 'penalty': 'l1'},
  mean: -0.92370, std: 0.04491, params: {'C': 2, 'penalty': 'l2'},
  mean: -0.88828, std: 0.06379, params: {'

In [ ]:
#model with parameters found on gridsearch

In [78]:
def stacker_gen(X,X_test,y,classifier):
    kf = StratifiedKFold(n_splits=5, random_state=26, shuffle=True)
    fold = 0
    y_test=0
    for train_index, test_index in kf.split(X, y):
        
        fold += 1

        X_train, X_valid    = X[train_index],   X[test_index]
        y_train, y_valid    = y[train_index],   y[test_index]

        print("Fold", fold, X_train.shape, X_valid.shape)

        clf=classifier
        clf.fit(X_train,y_train)
        p_test = clf.predict_proba(X_test)
        y_test += p_test/5
    classes = "class1,class2,class3,class4,class5,class6,class7,class8,class9".split(',')
    subm = pd.DataFrame(y_test, columns=classes)
    subm['ID'] = ID_test
    
    subm.to_csv("../final_scores/nw_stack_test.csv",index=False)
    
    print("cross_val sur train ") #peut etre que to array est exclusivement pour les xgb
    

    y_pred=cross_val_predict(estimator=clf,X=X,y=y,cv=kf,method="predict_proba")
    subm1 = pd.DataFrame(y_pred, columns=classes)
    subm1['ID'] = ID_train
    subm1.to_csv("../final_scores/nw_stack_train.csv",index=False)


In [79]:
# paramètre C et penalty à rentrer

In [82]:
clf = LogisticRegression(C=0.9,penalty="l1")
stacker_gen(X_train,X_test,y,clf)

Fold 1 (2946, 197) (743, 197)
Fold 2 (2949, 197) (740, 197)
Fold 3 (2952, 197) (737, 197)
Fold 4 (2954, 197) (735, 197)
Fold 5 (2955, 197) (734, 197)
cross_val sur train 
